In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../parameters.xml'

cfg = Parameters(config_file)

radius=0.02
longitude=0.2
time_range=[cfg.start_time, cfg.end_time]
use_grid=True
ylim_r=None
ylim_z=None
use_cache=True

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{E_r (\frac{V}{m})}$'
y_z_axis_label = r'$\mathit{E_z (\frac{V}{m})}$'

e_r_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Radial\enspace Component}\enspace(E_r)$'
e_z_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Longitudal\enspace Component}\enspace(E_z)$'
rho_beam_plot_name = r'$\mathbf{Electron\enspace Bunch\enspace Density}\enspace (\rho_{bunch})$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(cfg.data_path, cfg.system_state_path,
                         [cfg.number_r_grid , cfg.number_z_grid],
                         cfg.frames_per_file, True)
    reader.verbose = True
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [ ]:
# get data
start_frame = cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = cfg.get_frame_number_by_timestamp(time_range[1])
row_number = cfg.get_row_by_radius(radius)
col_number = cfg.get_col_by_longitude(longitude)

data_r = reader.get_frame_range_dot('E_r', row_number, col_number, start_frame, end_frame - 1)
data_z = reader.get_frame_range_dot('E_z', row_number, col_number, start_frame, end_frame - 1)

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data_r))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_r = plot.add_subplot_cartesian_2d(e_r_plot_name, 121, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)
plot_z = plot.add_subplot_cartesian_2d(e_z_plot_name, 122, x_axe_label=x_axis_label, y_axe_label=y_z_axis_label)

# set y-limits
if ylim_r is not None:
    plot_r.subplot_er.set_ylim(ylim_r)

if ylim_z is not None:
    plot_z.subplot_er.set_ylim(ylim_z)

In [ ]:
# add data
plot_r.plot(data_timeline, data_r)
plot_z.plot(data_timeline, data_z)

plot.show()